# TextGrid creation for mouse tracking data

## import modules and relevant functions from the evaluation notebook

In [3]:
%matplotlib inline
import pandas as pd
import matplotlib
import matplotlib.pyplot as plt
import numpy as np
from json import load
import os
import tgt

In [49]:
def find_file(suffix, path):
    """
    return the filename of the log file
    """
    result_files = []
    for root, dirs, files in os.walk(path):
        for file in [f for f in files if f.endswith(suffix)]:
            result_files.append(file)
    return result_files
        
def read_file(path):
    """
    return content of log file
    """
    with open(path) as raw_jfile:
        content = load(raw_jfile)
        return content
    
def crop_list(lst):
    """
    remove entries from log file before the game starts and after the game is finished
    return cropped file as list
    """
    sub_lst = []
    for entry in lst:
        try:
            if entry["msg"] == "Game started!" and entry["user"]["name"]=="Image_Click_Bot":
                sub_lst = lst[lst.index(entry):]
            if "no images left" in entry["msg"].lower() and entry["user"]["name"]=="Image_Click_Bot":
                sub_lst = lst[:lst.index(entry)+1]
        except:
            pass
    return sub_lst

def json_filename(file):
    """
    return file name for the json file used in the current log file
    """
    for entry in file: 
        try:
            if "json file:" in entry["msg"]:
                return (entry["msg"].split()[-1])
        except:
            pass

def images_in_log(file):
    """
    split log file on new_image commands
    return list containing one section for each image in log file, 
    each of them starting with a new_image command
    """
    file = crop_list(file)
    images = []
    i = False
    i_prev = False
    for entry in [e for e in file if e["type"] == "new_image"]:
        if i:
            i_prev = i
            i = file.index(entry)
            images.append(file[i_prev:i])
        else:
            i = file.index(entry)
    images.append(file[i:]) # slice from last new_image to end of file
    return images

In [158]:
def extract_data(img_slice):
    """
    extract data from img slice in log file:
    return timestamps for beginning and end, duration, tracking data and click with normalised timestamps
    """
    tracking_data = []
    clicks = []
    
    for entry in img_slice: 
        try: 
            if entry["type"]=="mouse_click" and entry["data"]["element"]== "#overlayButton":
                # get t_begin: timestamp from user click on overlay button
                t_begin = entry["data"]["timestamp"]
            if entry["type"]=="mouse_positions":
                tracking_data = entry["data"]
            if entry["type"] == "mouse_click":
                clicks.append({"x": entry["data"]["coordinates"]["x"],
                                   "y": entry["data"]["coordinates"]["y"],
                                   "element":entry["data"]["element"],
                                   "timestamp":entry["data"]["timestamp"]})
        except: 
            pass
    
    for entry in img_slice[::-1]:
        # last (i.e. correct) mouse click defines t_end
        if entry["type"] == "mouse_click":
            t_end = entry["data"]["timestamp"]
            break
        
    for entry in tracking_data + clicks:
        # set timestamps relative to t_begin
        entry["timestamp"] = entry["timestamp"] - t_begin
        
    duration = (t_end - t_begin)
        
    return {"tracking_data": tracking_data,"clicks": clicks,"t_begin": t_begin, "t_end": t_end, "duration": duration}

def data_and_filenames_from_log(file_path):
    """
    read log file and return list with extracted data from log slices and corresponding file names
    """
    
    cwd = os.getcwd()
    # switch to data path
    os.chdir(cwd+"/data/")
    
    # read content from log file
    log_content = read_file(file_path)
    
    # get filename from json file used in this session
    json_file = json_filename(log_content)
    
    json_file_content = read_file("json/"+json_file)
    filenames = []
    # iterate through entries in json_file_content
    for entry in json_file_content:
        try:
            # fetch filenames for audio (wav and json) and image files from log file
            audio_wav = log_json_content[entry]["audio_filename"]
            audio_json = os.path.splitext(audio_wav)[0]+".json"
            img_path = log_json_content[entry]["image_filename"]
            # append file names to "filenames" list
            filenames.append({'audio_wav':audio_wav, 'audio_json': audio_json, 'img': img_path})
        except:
            pass

    # split log file: separate parts for each image
    log_slices = images_in_log(log_file)
    # zip log sections with according image and audio file names (wav and json) 
    img_slices_with_paths = list(zip(log_slices, filenames))

    # extract data from image slice and pair with file names
    extracted_data_with_paths = [(extract_data(entry[0]),entry[1]) for entry in img_slices_with_paths]
    
    os.chdir(cwd)
    
    return (extracted_data_with_paths)


## read log files from the /data directory

In [164]:
cwd = os.getcwd()

In [165]:
os.chdir(cwd+"/data/")
files = find_file(".log", os.getcwd())
print (files)
os.chdir(cwd)

['2018-12-19 10-15-20-meetup 1.log']


## extract data

In [168]:
data_and_filenames_from_log(files[0])

[({'tracking_data': [{'timestamp': -7726054778772, 'x': 324, 'y': 617},
    {'timestamp': -7726054778762, 'x': 324, 'y': 617},
    {'timestamp': -7726054778740, 'x': 325, 'y': 616},
    {'timestamp': -7726054778661, 'x': 325, 'y': 616},
    {'timestamp': -7726054778638, 'x': 325, 'y': 617},
    {'timestamp': -7726054778626, 'x': 325, 'y': 617},
    {'timestamp': -7726054778603, 'x': 326, 'y': 617},
    {'timestamp': -7726054778591, 'x': 326, 'y': 617},
    {'timestamp': -7726054778546, 'x': 326, 'y': 618},
    {'timestamp': -7726054778525, 'x': 326, 'y': 618},
    {'timestamp': -7726054778514, 'x': 326, 'y': 618},
    {'timestamp': -7726054778493, 'x': 326, 'y': 619},
    {'timestamp': -7726054778470, 'x': 326, 'y': 619},
    {'timestamp': -7726054778460, 'x': 326, 'y': 619},
    {'timestamp': -7726054778438, 'x': 326, 'y': 619},
    {'timestamp': -7726054778428, 'x': 326, 'y': 619},
    {'timestamp': -7726054777992, 'x': 326, 'y': 618},
    {'timestamp': -7726054777981, 'x': 326, 'y':

In [75]:
# https://textgridtools.readthedocs.io/en/stable/api.html

# create new textgrid file
with open("test.textgrid", "w") as tg:
    tg.write('File type = "ooTextFile"\nObject class = "TextGrid"\n')
    
# define new textgrid object
textgrid = tgt.core.TextGrid(filename='test.textgrid')

# define tiers
clicks =  tgt.core.PointTier(start_time=0, end_time=img_duration/1000, name='clicks', objects=None)
movement = tgt.core.PointTier(start_time=0, end_time=img_duration/1000, name='movement', objects=None)
utterances = tgt.core.IntervalTier(start_time=0, end_time=img_duration/1000, name='utterances', objects=None)

# add tiers to textgrid object
textgrid.add_tiers((clicks, movement, utterances))

# write changes to file
tgt.write_to_file(textgrid, "test.textgrid", format='short')


# add data to textgrid
for entry in mouseclicks:
    # add mouseclicks to clicks tier
    clicks.add_point(tgt.core.Point(entry[0]/1000, text=entry[1]))
for entry in mouse_positions:
    # add tracking positions to movement tier
    movement.add_point(tgt.core.Point(entry['timestamp'], text=str([entry['x'], entry['y']])))
    
# save changes to file
tgt.write_to_file(textgrid, "test.textgrid", format='long')

In [39]:
# convert to elan format
elan = tgt.io.export_to_elan(textgrid, encoding='utf-8', include_empty_intervals=False, include_point_tiers=True, point_tier_annotation_duration=0.04)
with open("test.textgrid", "w") as tg:
    tg.write(elan)

In [7]:
cwd = "/home/simeon/Dokumente/mouse-tracking-experiment/evaluation/"
os.chdir(cwd)